In [2]:
! pip3 install datasets transformers torch

In [3]:
import numpy as np

In [4]:
from datasets import load_dataset

dataset = load_dataset("imdb", split="train")

np.random.seed(100)
idx = np.random.randint(len(dataset), size=200)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
len(dataset)

25000

In [53]:
from transformers import AutoTokenizer
from transformers import BertModel  # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel



def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']
    
    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }
    
    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }
    
    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [7]:
import torch
import torch.nn as nn
import numpy as np

from warnings import filterwarnings

filterwarnings('ignore')

In [38]:
device = torch.device('cpu')

In [39]:
from tqdm import tqdm


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    labels = []
    
    for batch in tqdm(loader):
        labels.append(batch['labels'].unsqueeze(1))

        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0), torch.cat(labels, dim=0).to(torch.float32)

In [36]:
tokenizer, model = get_model('bert')

In [10]:
def tokenization(example):
    return tokenizer.batch_encode_plus(example['text'], add_special_tokens=True, return_token_type_ids=False, truncation=True)

In [11]:
train_dataset = dataset.map(tokenization, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

NameError: name 'DataLoader' is not defined

In [13]:
from torch.utils.data import DataLoader

In [15]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
train_loader = DataLoader(torch.utils.data.Subset(train_dataset, idx), batch_size=64, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [23]:
train_embeddings, train_labels = get_embeddings_labels(model, train_loader)

100%|██████████| 4/4 [03:25<00:00, 51.32s/it]


In [24]:
train_embeddings

tensor([[ 0.6028,  0.1125, -0.2223,  ..., -0.1698,  0.1655,  0.0645],
        [ 0.6183,  0.0239, -0.2428,  ..., -0.1532,  0.1792,  0.1111],
        [ 0.3977,  0.1045, -0.1627,  ..., -0.0737,  0.2369,  0.1243],
        ...,
        [ 0.5241,  0.1857, -0.4136,  ..., -0.2503,  0.0959,  0.2350],
        [ 0.6121,  0.0789, -0.1658,  ..., -0.1467,  0.4310,  0.1626],
        [ 0.6343,  0.0062, -0.1479,  ..., -0.2116,  0.1337,  0.1664]])

In [27]:
train_embeddings.shape

torch.Size([200, 768])

In [28]:
torch.save(train_embeddings, 'bert_embeddings.pt')

In [54]:
tokenizer, model = get_model('roberta')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
def tokenization(example):
    return tokenizer.batch_encode_plus(example['text'], add_special_tokens=True, return_token_type_ids=False, truncation=True)

In [43]:
@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    labels = []
    
    for batch in tqdm(loader):
        labels.append(batch['labels'].unsqueeze(1))

        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0), torch.cat(labels, dim=0).to(torch.float32)

In [44]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [45]:
train_loader = DataLoader(torch.utils.data.Subset(train_dataset, idx), batch_size=64, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [46]:
train_embeddings, train_labels = get_embeddings_labels(model, train_loader)

100%|██████████| 4/4 [03:17<00:00, 49.27s/it]


In [47]:
train_embeddings.shape

torch.Size([200, 768])

In [48]:
torch.save(train_embeddings, 'roberta_embeddings.pt')

In [55]:
tokenizer, model = get_model('distilbert')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

In [56]:
def tokenization(example):
    return tokenizer.batch_encode_plus(example['text'], add_special_tokens=True, return_token_type_ids=False, truncation=True)

In [58]:
@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    labels = []
    
    for batch in tqdm(loader):
        labels.append(batch['labels'].unsqueeze(1))

        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0), torch.cat(labels, dim=0).to(torch.float32)

In [59]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [60]:
train_loader = DataLoader(torch.utils.data.Subset(train_dataset, idx), batch_size=64, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [61]:
train_embeddings, train_labels = get_embeddings_labels(model, train_loader)

100%|██████████| 4/4 [01:43<00:00, 25.80s/it]


In [62]:
train_embeddings.shape

torch.Size([200, 768])

In [63]:
torch.save(train_embeddings, 'distilbert_embeddings.pt')